In [ ]:
# Not necessary any more: Just use the files in the GitHub repo ~ Jakob: 28.11.2022 08:45

#!rm -rf waymo-od > /dev/null
#!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
#!cd waymo-od && git branch -a
#!cd waymo-od && git checkout remotes/origin/r1.0
#!pip3 install --upgrade pip

In [ ]:
# Those commands were only needed once. Just use the GitHub repo ~ Jakob: 28.11.2022 08:48

# protoc waymo_open_dataset/dataset.proto --python_out=.
# protoc waymo_open_dataset/label.proto --python_out=.

In [ ]:
# Imports 

#!pip3 install waymo-open-dataset

import os
import tensorflow as tf
import math
import numpy as np
import itertools
import matplotlib.pyplot as plt


from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset


# TODO: Use PyTorch instead of tensorflow

In [ ]:
# Loading the data sample data from the tutorial. ~ Jakob: 28.11.2022 09:36

#path = r'{pyglib_resource}data/'.format(pyglib_resource='')
path = r'{pyglib_resource}tutorial/frames'.format(pyglib_resource='')

dataset = tf.data.TFRecordDataset(path, compression_type='')
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break


In [ ]:
plt.figure(figsize=(14, 10))

def image_show(data, name, layout, cmap=None):
  """Show an image."""
  plt.subplot(*layout)
  plt.imshow(tf.image.decode_jpeg(data), cmap=cmap)
  plt.title(name)
  plt.grid(False)
  plt.axis('off')

for index, image in enumerate(frame.images):
  image_show(image.image, open_dataset.CameraName.Name.Name(image.name),
             [3, 3, index+1])

In [ ]:
# Not working yet. I tried to visualize the data of one of the training samples. But it didn't work out yet ~ Jakob 28.11.2022 09:43

from google.protobuf.json_format import MessageToJson
import json


dataset = tf.data.TFRecordDataset(r'data/training_20s.tfrecord-00000-of-01000')

for d in dataset:
    ex = tf.train.Example()
    ex.ParseFromString(d.numpy())
    m = json.loads(MessageToJson(ex))
    #print(m)

#for data in tf.python_io.tf_record_iterator(r'{pyglib_resource}training_20s'.format(pyglib_resource='')):
#    print(tf.train.Example.FromString(data))

In [ ]:
# Not working yet. ~ Jakob 28.11.2022 09:43

plt.figure(figsize=(64, 20))
def plot_range_image_helper(data, name, layout, vmin = 0, vmax=1, cmap='gray'):
  """Plots range image.

  Args:
    data: range image data
    name: the image title
    layout: plt layout
    vmin: minimum value of the passed data
    vmax: maximum value of the passed data
    cmap: color map
  """
  plt.subplot(*layout)
  plt.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax)
  plt.title(name)
  plt.grid(False)
  plt.axis('off')

def get_range_image(laser_name, return_index):
  """Returns range image given a laser name and its return index."""
  return range_images[laser_name][return_index]

def show_range_image(range_image, layout_index_start = 1):
  """Shows range image.

  Args:
    range_image: the range image data from a given lidar of type MatrixFloat.
    layout_index_start: layout offset
  """
  range_image_tensor = tf.convert_to_tensor(range_image.data)
  range_image_tensor = tf.reshape(range_image_tensor, range_image.shape.dims)
  lidar_image_mask = tf.greater_equal(range_image_tensor, 0)
  range_image_tensor = tf.where(lidar_image_mask, range_image_tensor,
                                tf.ones_like(range_image_tensor) * 1e10)
  range_image_range = range_image_tensor[...,0] 
  range_image_intensity = range_image_tensor[...,1]
  range_image_elongation = range_image_tensor[...,2]
  plot_range_image_helper(range_image_range.numpy(), 'range',
                   [8, 1, layout_index_start], vmax=75, cmap='gray')
  plot_range_image_helper(range_image_intensity.numpy(), 'intensity',
                   [8, 1, layout_index_start + 1], vmax=1.5, cmap='gray')
  plot_range_image_helper(range_image_elongation.numpy(), 'elongation',
                   [8, 1, layout_index_start + 2], vmax=1.5, cmap='gray')
frame.lasers.sort(key=lambda laser: laser.name)
show_range_image(get_range_image(open_dataset.LaserName.TOP, 0), 1)
show_range_image(get_range_image(open_dataset.LaserName.TOP, 1), 4)